For example:

```
162,817,812
57,618,57
906,360,560
592,479,940
352,342,300
466,668,158
542,29,236
431,825,988
739,650,466
52,470,668
216,146,977
819,987,18
117,168,530
805,96,715
346,949,466
970,615,88
941,993,340
862,61,35
984,92,344
425,690,689
```

This list describes the position of 20 junction boxes, one per line. Each position is given as X,Y,Z coordinates. So, the first junction box in the list is at X=162, Y=817, Z=812.

To save on string lights, the Elves would like to focus on connecting pairs of junction boxes that are as close together as possible according to straight-line distance. In this example, the two junction boxes which are closest together are 162,817,812 and 425,690,689.

By connecting these two junction boxes together, because electricity can flow between them, they become part of the same circuit. After connecting them, there is a single circuit which contains two junction boxes, and the remaining 18 junction boxes remain in their own individual circuits.

Now, the two junction boxes which are closest together but aren't already directly connected are 162,817,812 and 431,825,988. After connecting them, since 162,817,812 is already connected to another junction box, there is now a single circuit which contains three junction boxes and an additional 17 circuits which contain one junction box each.

The next two junction boxes to connect are 906,360,560 and 805,96,715. After connecting them, there is a circuit containing 3 junction boxes, a circuit containing 2 junction boxes, and 15 circuits which contain one junction box each.

The next two junction boxes are 431,825,988 and 425,690,689. Because these two junction boxes were already in the same circuit, nothing happens!

This process continues for a while, and the Elves are concerned that they don't have enough extension cables for all these circuits. They would like to know how big the circuits will be.

After making the ten shortest connections, there are 11 circuits: one circuit which contains 5 junction boxes, one circuit which contains 4 junction boxes, two circuits which contain 2 junction boxes each, and seven circuits which each contain a single junction box. Multiplying together the sizes of the three largest circuits (5, 4, and one of the circuits of size 2) produces 40.

In [2]:
data_test = """162,817,812
57,618,57
906,360,560
592,479,940
352,342,300
466,668,158
542,29,236
431,825,988
739,650,466
52,470,668
216,146,977
819,987,18
117,168,530
805,96,715
346,949,466
970,615,88
941,993,340
862,61,35
984,92,344
425,690,689"""

## Part 1

In [66]:
import numpy as np
from itertools import combinations
from tqdm import tqdm

np.set_printoptions(linewidth=110)
def data_to_3d(data):
    return np.array([list(map(int, line.split(","))) for line in data.splitlines()])
def calc_distances(pos):
    dist = []
    for i1, i2 in tqdm(list(combinations(range(len(pos)), 2))):
        dist.append(
            ( i1, i2, np.linalg.norm(pos[i1] - pos[i2]) )
        )
    return dist

In [185]:
def mk_circuits(pos, dist, n_connect = 10):
    circuits = []
    j2circ = {}
    for i in range(n_connect):
        j1, j2 = tuple(pos[dist[i][0]]), tuple(pos[dist[i][1]])
        # print(j1, j2)
        if not j1 in j2circ and not j2 in j2circ:
            circuits.append([j1, j2])
            circidx = len(circuits)-1
            print(f"joining in new circuit #{circidx}: {j1} {j2}")
            j2circ[j1] = circidx
            j2circ[j2] = circidx
        elif j1 in j2circ and not j2 in j2circ:
            circidx = j2circ[j1]
            print(f"joining pre-existing circuit #{circidx}: {j2}")
            circuits[circidx].append(j2)
            j2circ[j2] = circidx
        elif not j1 in j2circ and j2 in j2circ:
            circidx = j2circ[j2]
            print(f"joining pre-existing circuit #{circidx}: {j1}")
            circuits[circidx].append(j1)
            j2circ[j1] = circidx
        elif j1 in j2circ and j2 in j2circ:
            if j2circ[j1] == j2circ[j2]:
                print(f"skip. elements {j1} and {j2} already part of circuit {j2circ[j1]}")
                continue
            # everybody joins circuit1
            print(f"joining circuits {j2circ[j1]} (n={len(circuits[j2circ[j1]])}) and {j2circ[j2]} (n={len(circuits[j2circ[j2]])}) in {j2circ[j1]}")
            circid_j2 = j2circ[j2]
            circuits[j2circ[j1]].extend(circuits[circid_j2])
            for j in circuits[circid_j2]:
                j2circ[j] = j2circ[j1]
            circuits[circid_j2] = [] 
            print(f"new len of circuit {j2circ[j1]} (n={len(circuits[j2circ[j1]])})")

        else:
            assert False
    # return circuits
    return (
        circuits + 
        [[tuple(p)] for p in pos if not tuple(p) in j2circ]
    )


In [186]:
from functools import reduce

pos = data_to_3d(data_test)
dist = calc_distances(pos)
dist = sorted(dist, key = lambda x: x[2])
circuits = mk_circuits(pos, dist, n_connect = 10)
reduce(lambda x,y: x *y, sorted([len(c) for c in circuits], reverse=True)[:3])

100%|████████████████████████████████████████████████| 190/190 [00:00<00:00, 79899.51it/s]

joining in new circuit #0: (162, 817, 812) (425, 690, 689)
joining pre-existing circuit #0: (431, 825, 988)
joining in new circuit #1: (906, 360, 560) (805, 96, 715)
skip. elements (431, 825, 988) and (425, 690, 689) already part of circuit 0
joining in new circuit #2: (862, 61, 35) (984, 92, 344)
joining in new circuit #3: (52, 470, 668) (117, 168, 530)
joining in new circuit #4: (819, 987, 18) (941, 993, 340)
joining pre-existing circuit #1: (739, 650, 466)
joining pre-existing circuit #0: (346, 949, 466)
joining circuits 1 (n=3) and 2 (n=2) in 1
new len of circuit 1 (n=5)


40

In [190]:
from aocd import get_data
data = get_data(day=8, year=2025)
len(data.splitlines())

1000

In [189]:
from functools import reduce

pos = data_to_3d(data)
dist = calc_distances(pos)
dist = sorted(dist, key = lambda x: x[2])
circuits = mk_circuits(pos, dist, n_connect = 1000)
reduce(lambda x,y: x *y, sorted([len(c) for c in circuits], reverse=True)[:3])


100%|█████████████████████████████████████████| 499500/499500 [00:00<00:00, 534642.48it/s]

joining in new circuit #0: (22662, 6955, 7124) (22772, 6437, 7418)
joining in new circuit #1: (920, 52747, 91976) (770, 52177, 91358)
joining in new circuit #2: (42553, 71880, 29425) (43033, 72228, 28791)
joining in new circuit #3: (15380, 19660, 32704) (14656, 18849, 33244)
joining in new circuit #4: (37906, 65907, 6058) (37722, 67135, 5895)
joining in new circuit #5: (81678, 82926, 23052) (82215, 83791, 22210)
joining in new circuit #6: (2063, 92337, 39347) (3334, 92501, 40239)
joining in new circuit #7: (63502, 18502, 70887) (64925, 17822, 70885)
joining in new circuit #8: (45307, 39806, 4155) (46489, 40376, 2913)
joining in new circuit #9: (92384, 71229, 4904) (93976, 70443, 5258)
joining in new circuit #10: (66381, 11590, 38596) (64584, 11393, 39148)
joining in new circuit #11: (10466, 12639, 4121) (9446, 12943, 2526)
joining in new circuit #12: (96149, 7722, 78223) (94776, 6539, 78851)
joining in new circuit #13: (97992, 22128, 48571) (99950, 22091, 48206)
joining in new circuit 

62186

## Part 2 

In [211]:
def fully_connected(circuits):
    return len([c for c in circuits if c]) == 1 and len([c for c in circuits if c][0]) == len(pos)
    
def mk_circuits_full(pos, dist):
    circuits = []
    j2circ = {}

    for i in tqdm(range(len(dist))):    
        j1, j2 = tuple(pos[dist[i][0]]), tuple(pos[dist[i][1]])
        # print(j1, j2)
        if not j1 in j2circ and not j2 in j2circ:
            circuits.append([j1, j2])
            circidx = len(circuits)-1
            # print(f"joining in new circuit #{circidx}: {j1} {j2}")
            j2circ[j1] = circidx
            j2circ[j2] = circidx
        elif j1 in j2circ and not j2 in j2circ:
            circidx = j2circ[j1]
            # print(f"joining pre-existing circuit #{circidx}: {j2}")
            circuits[circidx].append(j2)
            j2circ[j2] = circidx
        elif not j1 in j2circ and j2 in j2circ:
            circidx = j2circ[j2]
            # print(f"joining pre-existing circuit #{circidx}: {j1}")
            circuits[circidx].append(j1)
            j2circ[j1] = circidx
        elif j1 in j2circ and j2 in j2circ:
            if j2circ[j1] == j2circ[j2]:
                # print(f"skip. elements {j1} and {j2} already part of circuit {j2circ[j1]}")
                continue
            # everybody joins circuit1
            # print(f"joining circuits {j2circ[j1]} (n={len(circuits[j2circ[j1]])}) and {j2circ[j2]} (n={len(circuits[j2circ[j2]])}) in {j2circ[j1]}")
            circid_j2 = j2circ[j2]
            circuits[j2circ[j1]].extend(circuits[circid_j2])
            for j in circuits[circid_j2]:
                j2circ[j] = j2circ[j1]
            circuits[circid_j2] = [] 
            # print(f"new len of circuit {j2circ[j1]} (n={len(circuits[j2circ[j1]])})")
        else:
            assert False
    
        if fully_connected(circuits):
            print(i, "full connect at", j1, j2)
            return j1, j2
            break
    return None, None


In [216]:
pos = data_to_3d(data)
dist = calc_distances(pos)
dist = sorted(dist, key = lambda x: x[2])
j1, j2 = mk_circuits_full(pos, dist)
if j1:
    print(j1[0] * j2[0])

  1%|▌                                          | 6277/499500 [00:00<00:01, 281278.27it/s]

6277 full connect at (86005, 23974, 12506) (97906, 28543, 4213)
8420405530
